In [6]:

import bempp.api 
bempp.api

import numpy as np
from bempp.api.operators.boundary.sparse import identity
from bempp.api.assembly.blocked_operator import BlockedOperator, GeneralizedBlockedOperator

bempp.api.enable_console_logging()
k = 1
h = 0.8

grid = bempp.api.shapes.sphere(h=h)
rwg = bempp.api.function_space(grid, 'RWG', 0)
rwg = bempp.api.function_space(grid, 'RWG', 0)
snc = bempp.api.function_space(grid, 'SNC', 0)


efie = bempp.api.operators.boundary.maxwell

bempp:HOST:INFO: Created grid with id 625296f5-5856-440c-81e4-651ba08eb5e0. Elements: 40. Edges: 60. Vertices: 22


In [2]:
bempp.api

<module 'bempp.api' from '/usr/local/lib/python3.9/dist-packages/bempp/api/__init__.py'>

In [4]:

ne1 = grid.number_of_edges
print(ne1, 'ne1')
print(M * ne1 * 2, 'Ndofs')
segments = [[10], [10]]
swapped_normals = [[10], []]
#swapped_normals = [[1,2,3,4,5,7,8,9,10,11],{}, [6]]
interface = [[10]]

bempp:HOST:INFO: Created grid with id 1caf9f67-b535-4935-8298-f5bbcb305dd6. Elements: 7108. Edges: 10662. Vertices: 3556


10662 ne1
42648 Ndofs


In [5]:
k_list = [k0] 
mu_list = [mu0]
for index in range(M-1):
  k_list.append(refractivei[index] * k0)
  mu_list.append(mui[index])

print(k_list, 'k_list')
    
dA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
rA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
tA = [bempp.api.function_space(grid, "SNC", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

[6.29, 12.599999999999998] k_list


In [6]:
multitrace_ops = []
# > Assemble all diagonal operators
for index in range(M):
  k = k_list[index]
  mu = mu_list[index]
  efie = bempp.api.operators.boundary.maxwell.electric_field(dA[index], rA[index], tA[index], k)
  mfie = bempp.api.operators.boundary.maxwell.magnetic_field(dA[index], rA[index], tA[index], k)
  multitrace_ops.append(bempp.api.GeneralizedBlockedOperator([[mfie, mu/k * efie],[- k/mu * efie, mfie]]))

# Define the final operator

block_system = [M * [None] for _ in range(M)]

for i in range(M):
  for j in range(M):
    if i == j:
      block_system[i][j] = 2 * multitrace_ops[i]
    else:
      all = segments[i] + segments[j]
      non_disjoint = np.unique(all).shape[0] != len(all)
      
      if non_disjoint:
        ident = identity(dA[j], rA[i], tA[i])
        op = BlockedOperator(2, 2)
        #op[0, 0] = -ident
        op[0, 0] = ident
        op[1, 1] = ident
        op.weak_form()
        #op[1, 1] = ident
        block_system[i][j] = op
      else:
        op = BlockedOperator(2, 2)
        zero = ZeroBoundaryOperator(dA[j], rA[i], tA[i])
        op[0, 0] = zero
        op[1, 1] = zero
        block_system[i][j] = op

block_system = GeneralizedBlockedOperator(block_system)


In [7]:

def incident_field_z(x):
    return np.array([0. * x[0], 0. * x[0], np.exp(1j * k0 * x[0])])


@bempp.api.complex_callable
def dirichlet_trace_z(x, n, domain_index, result):
    incident_field_z = np.array([0. * x[0], 0. * x[0], np.exp(1j * k0 * x[0])])
    result[:] = np.cross(incident_field_z, n)


@bempp.api.complex_callable
def neumann_trace_z(x, n, domain_index, result):
    curl_z = np.array([x[0] * 0. ,-1j * k0 * np.exp(1j * k0 * x[0]), x[0] * 0.])
#     with objmode():
    result[:] = (1/(1j * k0)) * np.cross(curl_z, n)

rhs = [ -2 * bempp.api.GridFunction(rA[0], dual_space = tA[0], fun=dirichlet_trace_z),
        -2 * k0 / mu0 * bempp.api.GridFunction(rA[0], dual_space = tA[0], fun=neumann_trace_z)]
for i in range(1, M):
    zero_func = [bempp.api.GridFunction.from_zeros(dA[i]),bempp.api.GridFunction.from_zeros(dA[i])]
    rhs = rhs + zero_func



In [8]:
block_system_wf = block_system.weak_form()


bempp:HOST:INFO: OpenCL CPU Device set to: pthread-AMD EPYC 7302 16-Core Processor


In [ ]:
print('weak_form')
block_system_wf = block_system.weak_form()
print('matrix')
A = bempp.api.as_matrix(block_system_wf)
print(A.shape, 'Ashape')
print('direct solver')
b = bempp.api.assembly.blocked_operator.projections_from_grid_functions_list(rhs, block_system.dual_to_range_spaces)
x = np.linalg.solve(A, b)
sol = bempp.api.assembly.blocked_operator.grid_function_list_from_coefficients(x.ravel(), block_system.domain_spaces)


weak_form
matrix
(42648, 42648) Ashape
direct solver


We compute the Far-Field pattern:
    

In [ ]:
number_of_angles = 3601
angles = np.pi * np.linspace(0, 2, number_of_angles)
unit_points = np.array([np.cos(angles), np.sin(angles), np.zeros(number_of_angles)])


electric_far = bempp.api.operators.far_field.maxwell.electric_field(sol[1].space, unit_points, k0)
magnetic_far = bempp.api.operators.far_field.maxwell.magnetic_field(sol[0].space, unit_points, k0)    

far_field = - electric_far * sol[1] * mu0 / k0 - magnetic_far * sol[0]

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure()
plt.rcParams['figure.figsize'] = (15, 10) # Increase the figure size in the notebook
params = {'legend.fontsize': 'x-large', 'axes.labelsize': 'x-large', 'axes.titlesize': 'x-large',
         'xtick.labelsize': 'x-large', 'ytick.labelsize': 'x-large'}
plt.rcParams.update(params)

bistatic_rcs= 10 * np.log10(4 * np.pi * np.sum(np.abs(far_field)**2, axis=0))
plt.plot(angles * 180 / np.pi, bistatic_rcs)

plt.title("Bistatic RCS [dB]")
_ = plt.xlabel('Angle (Degrees)')

In [ ]:
x.shape